### The benchmark of SARCOS dataset

The performance of different algorithms on this dataset can be found in: https://github.com/Kaixhin/SARCOS

#### Ridge regression with polynomial+fourier feature: 3.360